In [34]:
from utils import *
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

In [3]:
config = read_config('../config/main_config.yml')
print(config)

{'inputs': '../inputs/', 'outputs': '../outputs/', 'selenium_reqs': {'do_scape': False, 'driver_path': '/Users/shubhamjuneja/vscode/personal_projects/webscraping/selenium/chromedriver-mac-x64 3/chromedriver', 'website_path': 'https://finance.yahoo.com', 'saving_outputs': '../outputs/output_from_webscrape/', 'downloaded_file_location': '/Users/shubhamjuneja/Downloads', 'op_path_n_name': '../outputs/output_from_webscrape/company_to_symbol_mapping.csv'}, 'yfinance_reqs': {'input_path': '../inputs/input_for_yfinance', 'op_path': '../outputs/output_from_yfinance', 'nse_file_name': '../outputs/output_from_webscrape/MW-NIFTY-50-16-Jun-2024.csv'}, 'modeling_reqs': {'m_input_path': '../inputs/input_for_modeling'}}


In [27]:
mappings = pd.read_csv(config['yfinance_reqs']['nse_file_name'])
mappings.head()
data = pd.read_csv(os.path.join(config['yfinance_reqs']['op_path'],mappings['Symbols'][2]+'.csv'))
data['Date'] = pd.to_datetime(data['Date'])


In [28]:
mappings['Symbols'][2]

'ADANIPORTS'

In [29]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2007-11-27,154.000000,207.000000,154.000000,191.800003,174.525635,27262365
1,2007-11-28,194.000000,197.800003,174.929993,177.000000,161.058578,21958915
2,2007-11-29,181.000000,182.800003,168.460007,177.399994,161.422562,25616715
3,2007-11-30,178.000000,191.399994,178.000000,185.800003,169.066055,23047035
4,2007-12-03,187.949997,198.860001,184.399994,196.000000,178.347351,14578850


In [30]:
data['50_Moving_Avg'] = data['Close'].rolling(window=50).mean()
data['200_Moving_Avg'] = data['Close'].rolling(window=200).mean()

#data.sort_values(by='Date',ascending=False)

In [56]:

potential_buy=data.loc[data['Close']<data['50_Moving_Avg'],].sort_values(by='Date',ascending=False).reset_index(drop=True)

req_date_id = data.loc[data['Date'].isin(potential_buy['Date']),].index-1
potential_buy.columns = 'buying_date_'+potential_buy.columns 
req_dates_data=data.iloc[req_date_id,].sort_values(by='Date',ascending=False).reset_index(drop=True)
final_data = pd.concat([potential_buy,req_dates_data],axis = 1)
final_data.loc[(final_data['buying_date_Close']<final_data['buying_date_50_Moving_Avg']) &
               (final_data['Close']>final_data['50_Moving_Avg']),][['buying_date_Date','Date','buying_date_Close','Close','buying_date_50_Moving_Avg','50_Moving_Avg']]
# potential_buy.columns = potential_buy.columns +'_current_lesser'
# potential_buy['Before_date'] = potential_buy['Date_current_lesser'] - timedelta(days=1)
# potential_buy.merge(data[['Date','Close','50_Moving_Avg','200_Moving_Avg']],left_on='Before_date',right_on='Date',how='left')

,buying_date_Date,Date,buying_date_Close,Close,buying_date_50_Moving_Avg,50_Moving_Avg
0,2024-06-04,2024-06-03,1248.949951,1583.949951,1340.157000,1340.510000
6,2024-05-06,2024-05-03,1289.949951,1320.300049,1317.650999,1317.988999
7,2024-04-18,2024-04-16,1295.550049,1304.349976,1305.676997,1304.142996
9,2024-03-19,2024-03-18,1241.199951,1266.599976,1250.353992,1247.993992
10,2024-03-13,2024-03-12,1209.550049,1301.300049,1236.054993,1232.350991
...,...,...,...,...,...,...
1474,2009-07-06,2009-07-03,107.970001,117.169998,110.457599,109.912199
1499,2009-02-24,2009-02-20,69.239998,70.250000,69.871000,69.547200
1500,2009-02-18,2009-02-17,66.830002,69.190002,68.893000,68.606600
1570,2008-09-15,2008-09-12,107.150002,112.550003,111.410400,111.040800
